In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras.models import Model
from keras.layers import Input, Dense, ReLU
from keras import optimizers, layers
from tensorflow import math
import warnings
warnings.filterwarnings('ignore')

In [2]:
rain = pd.read_csv("./Data/Rainfall.csv")

In [3]:
import glob

slp_path  = "./Data/slp/*"
sst_path  = "./Data/sst/*"
uwnd_path = "./Data/uwnd/*"

slp_months = []
for i in glob.glob(slp_path):
  slp_months.append(i)

sst_months = []
for i in glob.glob(sst_path):
  sst_months.append(i)

uwnd_months = []
for i in glob.glob(uwnd_path):
  uwnd_months.append(i)
  
slp_months.sort()
sst_months.sort()
uwnd_months.sort()

In [4]:
#Here i am just storing rainfall data of Jun to sep from 1949 to 2014
# index 47= 1948,
import statistics
lpa = statistics.mean(list(rain['Jun-Sep'][60:89]))
print(lpa)
rain_fall_data = list(rain['Jun-Sep'][47:2020])
rain_data = []
for i in range(len(rain_fall_data)):
  temp = (rain_fall_data[i]/lpa)*100
  rain_data.append(temp)

878.0689655172414


In [5]:
from scipy.stats import pearsonr
month_path = slp_months[1] #0th in dex means january
month_data = pd.read_csv(month_path)

In [6]:
# 1948-2000
slp_s,  slp_e  = 0,  53
uwnd_s, uwnd_e = 0,  53
sst_s,  sst_e  = 94, 147
rain_s, rain_e = 47, 100

In [7]:
def get_feature(data):
  new_data = []
  for i in range(len(data)):
    d = data.iloc[i]
    new_data.append(list(d[1:]))
  return np.array(new_data)

# Stacking Data of All Months
combine_slp, combine_sst, combine_uwnd = [], [], []
for i in range(12):
  month_path_slp, month_path_sst, month_path_uwnd = slp_months[i], sst_months[i], uwnd_months[i] 
  month_data_slp ,month_data_sst, month_data_uwnd = pd.read_csv(month_path_slp), pd.read_csv(month_path_sst), pd.read_csv(month_path_uwnd)
 
  feature_slp, feature_sst, feature_uwnd = get_feature(month_data_slp), get_feature(month_data_sst), get_feature(month_data_uwnd)
  feature_slp, feature_sst, feature_uwnd = feature_slp[slp_s:slp_e], feature_sst[sst_s:sst_e], feature_uwnd[uwnd_s:uwnd_e]
  for i in feature_slp:
    combine_slp.append(i)
  for i in feature_sst:
    combine_sst.append(i)
  for i in feature_uwnd:
    combine_uwnd.append(i)
x_slp = np.array(combine_slp)
x_sst = np.array(combine_sst)
x_uwnd = np.array(combine_uwnd)
x = np.concatenate((x_slp, x_sst, x_uwnd), axis=1)
x.shape, x_slp.shape, x_sst.shape, x_uwnd.shape

((636, 713), (636, 324), (636, 65), (636, 324))

In [8]:
def get_lr(initial_learning_rate=0.001, decay_steps=4000, decay_rate=1):
    return optimizers.schedules.ExponentialDecay(
            initial_learning_rate,
            decay_steps=decay_steps,
            decay_rate=decay_rate,
            staircase=True)

In [9]:
def get_model_1(n_inputs=713):
  visible = Input(shape=(n_inputs,))
  e = Dense(n_inputs, kernel_regularizer=keras.regularizers.L2(0.01))(visible)
  e = ReLU()(e)
  bottleneck = Dense(97, kernel_regularizer=keras.regularizers.L2(0.01))(e)
  output = Dense(n_inputs, activation='linear', kernel_regularizer=keras.regularizers.L2(0.01))(bottleneck)
  model = Model(inputs=visible, outputs=output)
  model.compile(optimizer='adam', loss='mse')
  return model

In [10]:
model_1 = get_model_1()
model_1.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 713)]             0         
                                                                 
 dense (Dense)               (None, 713)               509082    
                                                                 
 re_lu (ReLU)                (None, 713)               0         
                                                                 
 dense_1 (Dense)             (None, 97)                69258     
                                                                 
 dense_2 (Dense)             (None, 713)               69874     
                                                                 
Total params: 648,214
Trainable params: 648,214
Non-trainable params: 0
_________________________________________________________________


In [11]:
history = model_1.fit(x, x, epochs=8000, batch_size=16, verbose=2, validation_data=(x,x))

Epoch 1/8000
40/40 - 1s - loss: 14572.7988 - val_loss: 9709.3730 - 1s/epoch - 36ms/step
Epoch 2/8000
40/40 - 0s - loss: 8357.5830 - val_loss: 6724.0435 - 467ms/epoch - 12ms/step
Epoch 3/8000
40/40 - 0s - loss: 6076.0269 - val_loss: 5104.1753 - 449ms/epoch - 11ms/step
Epoch 4/8000
40/40 - 0s - loss: 4795.4121 - val_loss: 4180.2261 - 434ms/epoch - 11ms/step
Epoch 5/8000
40/40 - 0s - loss: 4037.3774 - val_loss: 3571.9646 - 422ms/epoch - 11ms/step
Epoch 6/8000
40/40 - 0s - loss: 3524.2578 - val_loss: 3164.8572 - 449ms/epoch - 11ms/step
Epoch 7/8000
40/40 - 0s - loss: 3140.2087 - val_loss: 2859.1567 - 430ms/epoch - 11ms/step
Epoch 8/8000
40/40 - 1s - loss: 2867.6208 - val_loss: 2609.4539 - 501ms/epoch - 13ms/step
Epoch 9/8000
40/40 - 0s - loss: 2641.2390 - val_loss: 2417.5710 - 492ms/epoch - 12ms/step
Epoch 10/8000
40/40 - 0s - loss: 2490.4854 - val_loss: 2309.2664 - 467ms/epoch - 12ms/step
Epoch 11/8000
40/40 - 0s - loss: 2396.4783 - val_loss: 2185.8677 - 476ms/epoch - 12ms/step
Epoch 12/8

In [12]:
model_temp1 = Model(inputs=model_1.input, outputs=model_1.layers[3].output)
model_2_inputs = model_temp1(x)
model_2_inputs.shape

TensorShape([636, 97])

In [13]:
def get_model_2(n_inputs=97):
  visible = Input(shape=(n_inputs,))
  e = Dense(n_inputs, kernel_regularizer=keras.regularizers.L2(0.01))(visible)
  e = ReLU()(e)
  bottleneck = Dense(29, kernel_regularizer=keras.regularizers.L2(0.01))(e)
  output = Dense(n_inputs, activation='linear', kernel_regularizer=keras.regularizers.L2(0.01))(bottleneck)
  model = Model(inputs=visible, outputs=output)
  model.compile(optimizer='adam', loss='mse')
  return model

In [14]:
model_2 = get_model_2()
model_2.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 97)]              0         
                                                                 
 dense_3 (Dense)             (None, 97)                9506      
                                                                 
 re_lu_1 (ReLU)              (None, 97)                0         
                                                                 
 dense_4 (Dense)             (None, 29)                2842      
                                                                 
 dense_5 (Dense)             (None, 97)                2910      
                                                                 
Total params: 15,258
Trainable params: 15,258
Non-trainable params: 0
_________________________________________________________________


In [15]:
history = model_2.fit(model_2_inputs, model_2_inputs, epochs=8000, batch_size=16, verbose=2, validation_data=(model_2_inputs,model_2_inputs))

Epoch 1/8000
40/40 - 1s - loss: 32247.5684 - val_loss: 28204.7637 - 633ms/epoch - 16ms/step
Epoch 2/8000
40/40 - 0s - loss: 26679.9336 - val_loss: 24872.7012 - 68ms/epoch - 2ms/step
Epoch 3/8000
40/40 - 0s - loss: 23745.3750 - val_loss: 22213.8594 - 69ms/epoch - 2ms/step
Epoch 4/8000
40/40 - 0s - loss: 21276.7246 - val_loss: 19944.2539 - 69ms/epoch - 2ms/step
Epoch 5/8000
40/40 - 0s - loss: 19225.9395 - val_loss: 18142.3047 - 52ms/epoch - 1ms/step
Epoch 6/8000
40/40 - 0s - loss: 17667.0234 - val_loss: 16849.4062 - 71ms/epoch - 2ms/step
Epoch 7/8000
40/40 - 0s - loss: 16542.5312 - val_loss: 15846.6035 - 68ms/epoch - 2ms/step
Epoch 8/8000
40/40 - 0s - loss: 15635.9736 - val_loss: 15052.7295 - 70ms/epoch - 2ms/step
Epoch 9/8000
40/40 - 0s - loss: 14903.4951 - val_loss: 14390.1914 - 57ms/epoch - 1ms/step
Epoch 10/8000
40/40 - 0s - loss: 14309.6113 - val_loss: 13846.9014 - 68ms/epoch - 2ms/step
Epoch 11/8000
40/40 - 0s - loss: 13816.4717 - val_loss: 13404.3018 - 77ms/epoch - 2ms/step
Epoch 

In [16]:
model_temp2 = Model(inputs=model_2.input, outputs=model_2.layers[3].output)
model_3_inputs = model_temp2(model_2_inputs)
model_3_inputs.shape

TensorShape([636, 29])

In [17]:
def get_model_3(n_inputs=29):
  visible = Input(shape=(n_inputs,))
  e = Dense(n_inputs, kernel_regularizer=keras.regularizers.L2(0.01))(visible)
  e = ReLU()(e)
  bottleneck = Dense(9, kernel_regularizer=keras.regularizers.L2(0.01))(e)
  output = Dense(n_inputs, activation='linear', kernel_regularizer=keras.regularizers.L2(0.01))(bottleneck)
  model = Model(inputs=visible, outputs=output)
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mse')
  return model

In [18]:
model_3 = get_model_3()
model_3.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 29)]              0         
                                                                 
 dense_6 (Dense)             (None, 29)                870       
                                                                 
 re_lu_2 (ReLU)              (None, 29)                0         
                                                                 
 dense_7 (Dense)             (None, 9)                 270       
                                                                 
 dense_8 (Dense)             (None, 29)                290       
                                                                 
Total params: 1,430
Trainable params: 1,430
Non-trainable params: 0
_________________________________________________________________


In [19]:
history = model_3.fit(model_3_inputs, model_3_inputs, epochs=8000, batch_size=16, verbose=2, validation_data=(model_3_inputs,model_3_inputs))

Epoch 1/8000
40/40 - 1s - loss: 4285.4692 - val_loss: 3888.3215 - 548ms/epoch - 14ms/step
Epoch 2/8000
40/40 - 0s - loss: 3696.5947 - val_loss: 3504.2029 - 63ms/epoch - 2ms/step
Epoch 3/8000
40/40 - 0s - loss: 3390.5303 - val_loss: 3262.9536 - 61ms/epoch - 2ms/step
Epoch 4/8000
40/40 - 0s - loss: 3175.9297 - val_loss: 3076.8501 - 61ms/epoch - 2ms/step
Epoch 5/8000
40/40 - 0s - loss: 3002.3938 - val_loss: 2912.8601 - 69ms/epoch - 2ms/step
Epoch 6/8000
40/40 - 0s - loss: 2850.0247 - val_loss: 2773.0459 - 63ms/epoch - 2ms/step
Epoch 7/8000
40/40 - 0s - loss: 2719.9490 - val_loss: 2655.2122 - 61ms/epoch - 2ms/step
Epoch 8/8000
40/40 - 0s - loss: 2612.8020 - val_loss: 2558.6860 - 59ms/epoch - 1ms/step
Epoch 9/8000
40/40 - 0s - loss: 2524.9783 - val_loss: 2477.9058 - 64ms/epoch - 2ms/step
Epoch 10/8000
40/40 - 0s - loss: 2451.0562 - val_loss: 2411.2749 - 66ms/epoch - 2ms/step
Epoch 11/8000
40/40 - 0s - loss: 2390.1550 - val_loss: 2356.4165 - 62ms/epoch - 2ms/step
Epoch 12/8000
40/40 - 0s - l

In [20]:
def get_model_final(n_inputs=713):
  visible = Input(shape=(n_inputs,))
  e = Dense(n_inputs, kernel_regularizer=keras.regularizers.L2(0.01))(visible)
  e = ReLU()(e)
  e = Dense(97, kernel_regularizer=keras.regularizers.L2(0.01))(e)
  e = ReLU()(e)
  e = Dense(29, kernel_regularizer=keras.regularizers.L2(0.01))(e)
  e = ReLU()(e)
  bottleneck = Dense(9, kernel_regularizer=keras.regularizers.L2(0.01))(e)
  e = Dense(29, kernel_regularizer=keras.regularizers.L2(0.01))(bottleneck)
  e = ReLU()(e)
  e = Dense(97, kernel_regularizer=keras.regularizers.L2(0.01))(e)
  e = ReLU()(e)
  output = Dense(n_inputs, activation='linear', kernel_regularizer=keras.regularizers.L2(0.01))(e)
  model = Model(inputs=visible, outputs=output)
  model.compile(optimizer='adam', loss='mse')
  return model

In [21]:
model_final = get_model_final()
model_final.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 713)]             0         
                                                                 
 dense_9 (Dense)             (None, 713)               509082    
                                                                 
 re_lu_3 (ReLU)              (None, 713)               0         
                                                                 
 dense_10 (Dense)            (None, 97)                69258     
                                                                 
 re_lu_4 (ReLU)              (None, 97)                0         
                                                                 
 dense_11 (Dense)            (None, 29)                2842      
                                                                 
 re_lu_5 (ReLU)              (None, 29)                0   

In [22]:
model_final.layers[2] = model_1.layers[3]
model_final.layers[11] = model_1.layers[4]
model_final.layers[4] = model_2.layers[3]
model_final.layers[9] = model_2.layers[4]
model_final.layers[6] = model_3.layers[3]
model_final.layers[7] = model_3.layers[4]
model_final.compile(optimizer='adam', loss='mse')

In [23]:
model_final.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 713)]             0         
                                                                 
 dense_9 (Dense)             (None, 713)               509082    
                                                                 
 re_lu_3 (ReLU)              (None, 713)               0         
                                                                 
 dense_10 (Dense)            (None, 97)                69258     
                                                                 
 re_lu_4 (ReLU)              (None, 97)                0         
                                                                 
 dense_11 (Dense)            (None, 29)                2842      
                                                                 
 re_lu_5 (ReLU)              (None, 29)                0   

In [24]:
history = model_final.fit(x, x, epochs=10000, batch_size=16, verbose=2, validation_data=(x,x))

Epoch 1/10000
40/40 - 2s - loss: 16267.4307 - val_loss: 13227.8320 - 2s/epoch - 43ms/step
Epoch 2/10000
40/40 - 0s - loss: 12323.8037 - val_loss: 11510.9297 - 407ms/epoch - 10ms/step
Epoch 3/10000
40/40 - 0s - loss: 11343.7236 - val_loss: 10853.8389 - 447ms/epoch - 11ms/step
Epoch 4/10000
40/40 - 0s - loss: 10748.2334 - val_loss: 10352.6982 - 455ms/epoch - 11ms/step
Epoch 5/10000
40/40 - 0s - loss: 10369.6035 - val_loss: 10046.5176 - 499ms/epoch - 12ms/step
Epoch 6/10000
40/40 - 0s - loss: 10127.6084 - val_loss: 9904.4961 - 424ms/epoch - 11ms/step
Epoch 7/10000
40/40 - 0s - loss: 9925.5947 - val_loss: 9687.2568 - 413ms/epoch - 10ms/step
Epoch 8/10000
40/40 - 0s - loss: 9725.8486 - val_loss: 9510.5479 - 462ms/epoch - 12ms/step
Epoch 9/10000
40/40 - 0s - loss: 9530.4502 - val_loss: 9218.7197 - 432ms/epoch - 11ms/step
Epoch 10/10000
40/40 - 0s - loss: 9207.6094 - val_loss: 8884.2441 - 404ms/epoch - 10ms/step
Epoch 11/10000
40/40 - 0s - loss: 8937.4922 - val_loss: 8682.9805 - 434ms/epoch -

In [25]:
model_final.save("./Data/Models/model_final_slp_sst_uwnd.h5")
# model_final = keras.models.load_model('./Data/Models/model_final.h5')

In [26]:
class Custom_Layer(layers.Layer):
    def __init__(self, weights, bias=False):
        super(Custom_Layer, self).__init__()
        self.w = weights
        if bias:
            self.b = bias

    def call(self, inputs):
        try: return tf.matmul(inputs, self.w) + self.b
        except: return tf.matmul(inputs, self.w)

In [27]:
std = []
for i in model_final.layers:
    # print(i.weights[0])
    try: std.append(np.array(math.reduce_std(i.weights[0])) * 2)
    except: std.append([])

In [28]:
final_layers = keras.Sequential([Input(shape=(None, 713))])
for i, layer in enumerate(model_final.layers):
    try:
        temp_layer = []
        for ind, j in enumerate(layer.weights[0]):
            temp_weights = []
            for index, k in enumerate(j):
                if k < std[i]: temp_weights.append(0)
                else:          temp_weights.append(1)  
            temp_layer.append(temp_weights)
        weights = tf.Variable(temp_layer, dtype='float32')
        try: final_layers.add(Custom_Layer(weights, bias=layer.weights[1]))
        except: final_layers.add(Custom_Layer(weights))
    except IndexError: final_layers.add(layer)

In [29]:
final_layers.compile(optimizer='adam', loss='mse')
final_layers.build(input_shape=(None, 713))
final_layers.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        multiple                  0         
                                                                 
 custom__layer_1 (Custom_Lay  (None, None, 713)        508369    
 er)                                                             
                                                                 
 re_lu_3 (ReLU)              multiple                  0         
                                                                 
 custom__layer_3 (Custom_Lay  (None, None, 97)         69161     
 er)                                                             
                                                                 
 re_lu_4 (ReLU)              multiple                  0         
                                                                 
 custom__layer_5 (Custom_Lay  (None, None, 29)         2

In [30]:
model_final1 = Model(inputs=model_final.input, outputs=model_final.layers[3].output)
model_final1.trainable = False

model_final2 = Model(inputs=model_final.input, outputs=model_final.layers[5].output)
model_final2.trainable = False

model_final3 = Model(inputs=model_final.input, outputs=model_final.layers[7].output)
model_final3.trainable = False

models = [model_final1, model_final2, model_final3]
# models = [model_final1, model_final4]

Note that input tensors are instantiated via `tensor = tf.keras.Input(shape)`.
The tensor that caused the issue was: KerasTensor(type_spec=TensorSpec(shape=(None, 713), dtype=tf.float32, name='input_4'), name='input_4', description="created by layer 'input_4'")
Note that input tensors are instantiated via `tensor = tf.keras.Input(shape)`.
The tensor that caused the issue was: KerasTensor(type_spec=TensorSpec(shape=(None, 713), dtype=tf.float32, name='input_4'), name='input_4', description="created by layer 'input_4'")
Note that input tensors are instantiated via `tensor = tf.keras.Input(shape)`.
The tensor that caused the issue was: KerasTensor(type_spec=TensorSpec(shape=(None, 713), dtype=tf.float32, name='input_4'), name='input_4', description="created by layer 'input_4'")


In [31]:
final_layers1 = Model(inputs=final_layers.input, outputs=final_layers.layers[3].output)
final_layers1.trainable = False

final_layers2 = Model(inputs=final_layers.input, outputs=final_layers.layers[5].output)
final_layers2.trainable = False

final_layers3 = Model(inputs=final_layers.input, outputs=final_layers.layers[7].output)
final_layers3.trainable = False

models = [final_layers1, final_layers2, final_layers3]

In [32]:
def get_feature(data):
  new_data = []
  for i in range(len(data)):
    d = data.iloc[i]
    new_data.append(list(d[1:]))
  # print(new_data)
  return np.array(new_data)

# to combine all the features together
def predictor(month,mod):
  # chacking for each MONTH
  month_path_slp, month_path_sst, month_path_uwnd = slp_months[i], sst_months[i], uwnd_months[i]
  # reading month data
  month_data_slp ,month_data_sst, month_data_uwnd = pd.read_csv(month_path_slp), pd.read_csv(month_path_sst), pd.read_csv(month_path_uwnd)
  feature_slp, feature_sst, feature_uwnd = get_feature(month_data_slp), get_feature(month_data_sst), get_feature(month_data_uwnd)
  feature_slp, feature_sst, feature_uwnd = feature_slp[slp_s:slp_s+72], feature_sst[sst_s:sst_s+72], feature_uwnd[uwnd_s:uwnd_s+72]
  x_slp = np.array(feature_slp)
  x_sst = np.array(feature_sst)
  x_uwnd = np.array(feature_uwnd)
  feature = np.concatenate((x_slp, x_sst, x_uwnd), axis=1)
  pred_m = models[mod](feature)

  pred_f = pred_m.numpy()
  return pred_f

In [33]:
from sklearn.svm import SVR


#w_start = window start, w_end = window end  
def get_top_pred(top,pred_info,w_start,w_end):
  k = 0
  pred_f = pred_info
  cor_list = []
  for i in range(len(pred_f[0])):
    score = []
    
    for j in range(w_start,w_end):
      score.append(pred_f[j][i])
    corr, _ = pearsonr(rain_fall_data[w_start:w_end],score)
    cor_list.append(corr)
    # print(corr)
  list1=list(enumerate(cor_list)) #I have taken cosine rank[0] beacuse it is inside the list
  list2=sorted(list1, key=lambda x: x[1],reverse=True)
  top_feature_index = []
  #here i am getting the top features index
  for i in range(top):
    index = list2[i][0]
    top_feature_index.append(index)
  # Here i am getting all the top features
  predictor = []
  for i in range(len(pred_f)):
    temp = []
    for j in top_feature_index:
      # print(i,"  ",j)
      feature = pred_f[i][j]
      temp.append(feature)
    predictor.append(temp)
  # print(list2)
  return predictor
  # print(list1)


# np.concatenate((a, b.T), axis=1)
def get_predictors(months,top,models,w_start,w_end):
  for i in months:
    for mod in range(len(models)):
      y_pred = predictor(i,mod)
      b = get_top_pred(top, y_pred, w_start,w_end)
      try:    pred = np.concatenate((pred,b), axis=1)
      except: pred = np.array(b)
  return pred

def window_solution(months,top):
  reg = SVR(kernel = 'rbf',C=1.0,epsilon=0.45)
  # reg = make_pipeline(StandardScaler(), SVR(kernel = 'rbf',C=1.0, epsilon=0.2))
  # months = [0]
  k=0
  cor_all = []
  for window in range(10,53):
    k+=1
    score = []
    pred = get_predictors(months,top,models,53-window,53)
    for i in range(14):
      start = 53-window
      end = 53+1
      # pred = get_predictors(months,2,models,start,end)
      reg.fit(pred[53-window:53+i], rain_fall_data[53-window:53+i])
      score.append(reg.predict([pred[53+i]])[0])
    # print(score)
    corr, _ = pearsonr(rain_fall_data[53:67],score)
    print("Window size = ",window,"   plcc",corr)
    cor_all.append(corr)
  return min(cor_all),max(cor_all)
#This is by taking correlation after every iteraion of each predict

In [34]:
from itertools import combinations
# months_comb = [j for i in range(4) for j in combinations(range(5,9), i+1)]
months_comb = [[4]]
top = 9
# models = [model_final1, model_final2, model_final3]
models = [final_layers1, final_layers2, final_layers3]

minimum, maximum = 1, -1
for month in months_comb:
  for i in range(1,top):
    left, right = window_solution(month,i)
    print("\n\nMonth combination = ",month,"    top = ",i,"   min and max",left,"   ",right)
    if minimum > left:  minimum, min_month, min_top = left, month, i
    if maximum < right: maximum, max_month, max_top = right, month, i
    print(f'Minimum: {minimum}, {min_month}, {min_top}    Maximum: {maximum}, {max_month}, {max_top}\n')

Window size =  10    plcc -0.29950230388260296
Window size =  11    plcc -0.49453251542281423
Window size =  12    plcc -0.5429453227593375
Window size =  13    plcc -0.43906710941820576
Window size =  14    plcc -0.5429453227593375
Window size =  15    plcc -0.49453251542281423
Window size =  16    plcc -0.29950230388260296
Window size =  17    plcc -0.49453251542281423
Window size =  18    plcc -0.547424700596782
Window size =  19    plcc -0.49453251542281423
Window size =  20    plcc -0.5519704491543065
Window size =  21    plcc -0.3220549282820413
Window size =  22    plcc -0.5519704491543065
Window size =  23    plcc -0.3220549282820413
Window size =  24    plcc -0.38515632836819513
Window size =  25    plcc -0.4804142999823574
Window size =  26    plcc -0.5115114674406023
Window size =  27    plcc -0.4804142999823574
Window size =  28    plcc -0.5115114674406023
Window size =  29    plcc -0.4804142999823574
Window size =  30    plcc -0.5115114674406023
Window size =  31    plcc -